In [15]:
import pandas as pd
from sklearn import metrics
from operator import itemgetter

In [23]:
df_pre = pd.read_csv('/mnt/isilon/cbmi/variome/perry/projects/sarmadi/mahdi_epi/work/roc_df_combo', sep='\t')
df = df_pre[df_pre.ccr!=-1]
df.head()

/opt/conda/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,chrom,pos,alt,mpc,y,Disease,revel,ccr,is_domain,mpc-revel_pred_lm,mpc-ccr_pred_lm,mpc-is_domain_pred_lm,revel-ccr_pred_lm,revel-is_domain_pred_lm,ccr-is_domain_pred_lm,mpc-revel-ccr_pred_lm,mpc-revel-is_domain_pred_lm,mpc-ccr-is_domain_pred_lm,revel-ccr-is_domain_pred_lm,mpc-revel-ccr-is_domain_pred_lm
0,13,31891746,A,0.3355,0,uc-epi,0.289,0.000,0,0.112411,0.045478,0.037441,0.109512,0.135816,0.029363,0.098877,0.114877,0.029608,0.110299,0.101172
1,14,31549832,A,0.1423,0,uc-epi,0.049,0.000,0,-0.056141,0.012501,-0.008532,-0.038392,-0.046927,0.029363,-0.046243,-0.054951,-0.001429,-0.038020,-0.045138
2,19,42728836,C,0.3706,0,uc-epi,0.037,0.000,0,-0.037250,0.051608,0.045823,-0.045728,-0.056164,0.029324,-0.035119,-0.036116,0.035210,-0.045483,-0.034095
3,19,42729121,T,0.3530,0,uc-epi,0.060,0.000,0,-0.025235,0.048604,0.041633,-0.031556,-0.038651,0.029324,-0.023993,-0.024004,0.032385,-0.031286,-0.022881
4,19,42730204,C,1.3121,0,uc-epi,0.550,46.789,1,0.383962,0.473268,0.351962,0.480019,0.335252,0.512215,0.485044,0.380028,0.509218,0.479117,0.481460


In [26]:
auc_ls = []
def eval_roc(df, disease, fout):
    scores = [x for x in df.columns.values if not x in ('chrom', 'pos', 'alt', 'Disease', 'y')]
    for score in scores:
        fpr, tpr, _ = metrics.roc_curve(df['y'], df[score], pos_label=1)
        auc = metrics.auc(fpr, tpr)
        auc_ls.append((auc, score, disease))
        for f, t in zip(fpr, tpr):
            print('\t'.join((disease, str(f), str(t), score)), file=fout)

diseases = set(df['Disease'])
with open('roc_eval', 'w') as fout:
    print('disease\tfpr\ttpr\tfeature', file=fout)
    for dis in diseases:
        if not 'earing' in dis and not 'issue' in dis:
            eval_roc(df[dis==df.Disease], dis, fout)
auc_df = pd.DataFrame(auc_ls, columns=['auc', 'features', 'disease'])
#sorted(auc_ls, key=itemgetter(0))

In [27]:
auc_df[auc_df.disease=='uc-epi'].sort_values(by='auc', ascending=False)

,auc,features,disease
52,0.982442,revel-ccr_pred_lm,uc-epi
58,0.982353,revel-ccr-is_domain_pred_lm,uc-epi
46,0.980688,revel,uc-epi
53,0.979836,revel-is_domain_pred_lm,uc-epi
55,0.979581,mpc-revel-ccr_pred_lm,uc-epi
59,0.979567,mpc-revel-ccr-is_domain_pred_lm,uc-epi
56,0.976149,mpc-revel-is_domain_pred_lm,uc-epi
49,0.976130,mpc-revel_pred_lm,uc-epi
57,0.885683,mpc-ccr-is_domain_pred_lm,uc-epi
50,0.862722,mpc-ccr_pred_lm,uc-epi


In [28]:
print(diseases)

{'Connective tissue disorders', 'Hearing Loss', 'uc-epi-limitGene', 'genedx-epi', 'Cardiomyopathy', 'uc-epi', 'Rasopathies', 'genedx-epi-limitGene'}


In [29]:
auc_df[auc_df.disease=='genedx-epi'].sort_values(by='auc', ascending=False)

,auc,features,disease
22,0.981173,revel-ccr_pred_lm,genedx-epi
28,0.980771,revel-ccr-is_domain_pred_lm,genedx-epi
25,0.980155,mpc-revel-ccr_pred_lm,genedx-epi
29,0.979957,mpc-revel-ccr-is_domain_pred_lm,genedx-epi
16,0.978491,revel,genedx-epi
19,0.977896,mpc-revel_pred_lm,genedx-epi
26,0.977505,mpc-revel-is_domain_pred_lm,genedx-epi
23,0.975027,revel-is_domain_pred_lm,genedx-epi
15,0.873007,mpc,genedx-epi
20,0.866231,mpc-ccr_pred_lm,genedx-epi


In [30]:
auc_df[auc_df.disease=='Cardiomyopathy'].sort_values(by='auc', ascending=False)

,auc,features,disease
34,0.995594,mpc-revel_pred_lm,Cardiomyopathy
41,0.995594,mpc-revel-is_domain_pred_lm,Cardiomyopathy
36,0.995530,mpc-is_domain_pred_lm,Cardiomyopathy
30,0.995464,mpc,Cardiomyopathy
42,0.984142,mpc-ccr-is_domain_pred_lm,Cardiomyopathy
35,0.980254,mpc-ccr_pred_lm,Cardiomyopathy
44,0.968494,mpc-revel-ccr-is_domain_pred_lm,Cardiomyopathy
40,0.964609,mpc-revel-ccr_pred_lm,Cardiomyopathy
38,0.933164,revel-is_domain_pred_lm,Cardiomyopathy
31,0.890398,revel,Cardiomyopathy


In [31]:
auc_df[auc_df.disease=='genedx-epi-limitGene'].sort_values(by='auc', ascending=False)

,auc,features,disease
82,0.987953,revel-ccr_pred_lm,genedx-epi-limitGene
85,0.987611,mpc-revel-ccr_pred_lm,genedx-epi-limitGene
89,0.987554,mpc-revel-ccr-is_domain_pred_lm,genedx-epi-limitGene
76,0.987454,revel,genedx-epi-limitGene
88,0.987326,revel-ccr-is_domain_pred_lm,genedx-epi-limitGene
86,0.986329,mpc-revel-is_domain_pred_lm,genedx-epi-limitGene
79,0.986244,mpc-revel_pred_lm,genedx-epi-limitGene
83,0.985959,revel-is_domain_pred_lm,genedx-epi-limitGene
80,0.937087,mpc-ccr_pred_lm,genedx-epi-limitGene
87,0.934866,mpc-ccr-is_domain_pred_lm,genedx-epi-limitGene
